# Deep Learning Models Experiments -  Premier Analysis on Azure Machine Learning
### LSTM and DAN 

In [51]:
import argparse
import os
import sklearn
import pandas as pd 
import numpy as np
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from azureml.core import Run, Dataset, Environment,Experiment,ScriptRunConfig
from sklearn.preprocessing import LabelEncoder
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

In [52]:
from azureml.core import  Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="9ce70869-60db-44fd-abe8-d2767077fc8f")

ws = Workspace.from_config()

In [53]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cdh-azml-dev-mlw
Azure region: eastus
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: csels-cdh-dev


In [54]:
# current working directory
path = os.getcwd()
print("Current Directory:", path)
  
# parent directory
parent = os.path.join(path, os.pardir)
  
# prints parent directory
print("\nParent Directory:", os.path.abspath(parent))

premier_path = os.path.abspath(parent)

Current Directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/azure_ml

Parent Directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis


### Create Compute

In [55]:
clustername = 'StandardNC6'
is_new_cluster = False
try:
    aml_cluster = ComputeTarget(workspace = ws,name= clustername)
    print("Find the existing cluster")
except ComputeTargetException:
    print("Cluster not find - Creating cluster.....")
    is_new_cluster = True
    compute_config = AmlCompute.provisioning_configuration(vm_size='StandardNC6',
                                                           max_nodes=2)
    aml_cluster = ComputeTarget.create(ws, clustername, compute_config)

aml_cluster.wait_for_completion(show_output=True)

Find the existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [56]:
features_path = os.path.join(premier_path,'output/parquet')
features_pkl = os.path.join(premier_path,'output/pkl')


data_store = ws.get_default_datastore()
data_store.upload(src_dir=features_path,target_path='parquet',overwrite=True,show_progress=True)
data_store.upload(src_dir=features_pkl,target_path='pkl',overwrite=True,show_progress=True)

Uploading an estimated of 3 files
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore.amltmp
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore.amltmp, 1 files out of an estimated total of 3
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/.amlignore, 2 files out of an estimated total of 3
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/flat_features.parquet
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/parquet/flat_features.parquet, 3 files out of an estimated total of 3
Uploaded 3 files
Uploading an estimated

$AZUREML_DATAREFERENCE_c5e76c8c994e418e902e99c15165eda7

In [57]:
cohort_path = os.path.join(premier_path,'output/cohort')
data_store.upload(src_dir=cohort_path,target_path='cohort',overwrite=True,show_progress=True)

Uploading an estimated of 3 files
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/cohort/.amlignore
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/cohort/.amlignore, 1 files out of an estimated total of 3
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/cohort/.amlignore.amltmp
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/cohort/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/cohort/cohort.csv
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/output/cohort/cohort.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_2b0e965bb5514ae8828de2421a73c39f

In [58]:
%%writefile conda_dependencies_model.yml

channels:
- anaconda
- default
dependencies:
- python=3.8
- pip:
  - azureml-defaults
  - matplotlib
  - pandas
  - argparse
  - joblib
  - scikit-learn
  - azureml-sdk
  - openpyxl
  - tensorflow
  - keras-tuner

Overwriting conda_dependencies_model.yml


In [59]:
premier_train_model_env = Environment.from_conda_specification(name='premier_train_model_env', file_path='conda_dependencies_model.yml')
# Specify a GPU base image
premier_train_model_env.docker.enabled = True
premier_train_model_env.docker.base_image = DEFAULT_CPU_IMAGE
premier_train_model_env.register(workspace=ws)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "premier_train_model_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "defa

In [64]:

mod_names = ['dan']
#outcomes = ['icu','death','misa_pt']
outcomes = ['misa_pt']
EPOCHS = 10
for outcome in outcomes:
    # Create experiment
    experiment = Experiment(workspace=ws, name=f"Premier-outcome-{outcome}")
    for mod in mod_names:

        estimator = ScriptRunConfig(source_directory='./training',
                            script='train_model.py',
                            compute_target=aml_cluster,
                            arguments=['--day_one','--outcome', outcome,'--model', mod,'--epochs',EPOCHS],
                            environment=premier_train_model_env)

        
        print("Submit Experiment")
        run = experiment.submit(estimator)

Submit Experiment


### Download Model

In [65]:
run.get_file_names()

['logs/azureml/dataprep/0/rslex.log.2022-10-04-16',
 'outputs/model/dan_d1_misa_pt/keras_metadata.pb',
 'outputs/model/dan_d1_misa_pt/saved_model.pb',
 'outputs/model/dan_d1_misa_pt/variables/variables.data-00000-of-00001',
 'outputs/model/dan_d1_misa_pt/variables/variables.index',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [62]:
# Create a model folder in the current directory
#os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        #run.download_file(name=f, output_file_path=output_file_path)

In [66]:
model = run.register_model(model_name='dan_d1_misa_pt', model_path=os.path.join('outputs/model','dan_d1_misa_pt'))
print(model.name, model.version, sep = '\t')

dan_d1_misa_pt	2


## Predict outcome

## Register model